# PREDICTING FUTURE SALES

Общие библиотеки

In [1]:
import pandas as pd                                      
import numpy as np                    
from scipy import optimize, stats 
import os

# visualisation libraries
import matplotlib.pyplot as plt                      
import seaborn as sns                

In [2]:
!pip install catboost

In [3]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU

import catboost as cb
from catboost import CatBoostRegressor

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import matplotlib.pyplot as plt

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [6]:
import time

In [7]:
df_train = pd.read_csv('sales_train.csv')
df_shops = pd.read_csv('shops.csv')
df_items = pd.read_csv('items.csv')
df_item_categories = pd.read_csv('item_categories.csv')
df_test = pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sales_train.csv'

**Prepare and clean dataset if nesessary**

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.value_counts

In [ ]:
df_train.shape

**Analysis of the dataset for anomalies: outliers, negative values, dependencies.**

In [ ]:
df_train["item_price"]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(df_train["item_price"], vert = 0)

# show plot
plt.show()

In [ ]:
fig = plt.figure(figsize =(10, 7))
# Creating plot
plt.boxplot(df_train["item_cnt_day"], vert = 0)
# show plot
plt.show()

In [ ]:
df_train.loc[df_train['item_price']>50000]

In [ ]:
df_train.drop(index=1163158, axis=0, inplace=True)

In [ ]:
df_sorted= df_train.sort_values(by="item_price")
df_sorted.tail(5)

In [ ]:
df_train.loc[df_train['item_price'] <0]

In [ ]:
df_train.drop(index=484683, axis=0, inplace=True)

In [ ]:

fig = plt.figure(figsize=(18,8))
plt.scatter(df_train["item_price"], df_train["item_cnt_day"])

In [ ]:
df_train.drop(index=2909818, axis=0, inplace=True)

In [ ]:
df_train.loc[df_train['item_cnt_day']<0]

In [ ]:
df_train.loc[df_train['item_cnt_day']>800]

**Check for empty and Nan values and clean if nesessary**


In [ ]:
df_test.shape

In [ ]:
df_test.describe()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.isna().sum()

**Group and join tables**

In [ ]:
df_train.drop(['date_block_num','item_price'], axis=1, inplace=True)

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'], dayfirst=True)
df_train['date'] = df_train['date'].apply(lambda x: x.strftime('%Y-%m'))
df_train.head(15)

In [ ]:
df = df_train.groupby(['date','shop_id','item_id']).sum()
df = df.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
df.reset_index(inplace=True)
df

In [ ]:
df_test = pd.merge(df_test, df, on=['shop_id','item_id'], how='left')
#df_test.drop(['ID', '2013-01', '2015-10'], axis=1, inplace=True)
df_test.drop(['2013-01', '2015-10'], axis=1, inplace=True)
id_col= df_test.pop("ID")
df_test = df_test.fillna(0)
df_test

In [ ]:
Y_train = df['2015-10'].values
X_train = df.drop(['2015-10', '2013-01'], axis = 1)
X_test = df_test

print(X_train.shape, Y_train.shape)
print(X_test.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.2, random_state=101)
# x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.2)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

In [ ]:
LR = LinearRegression()
LR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, LR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, LR.predict(x_test)))
print('Test set score:', LR.score(x_test,y_test))
print('Train set score:', LR.score(x_train,y_train))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))
fig.suptitle('Метрики качества с очисткой', size =16)

ax[0].bar(x="Train",height = mean_squared_error(y_train, LR.predict(x_train)), color='y', alpha=0.5)
ax[0].bar(x="Test", height= mean_squared_error(y_test, LR.predict(x_test)), color='r', alpha=0.5)
ax[0].set_title("MSE ")

ax[1].bar(x="Train", height= LR.score(x_test,y_test), color='y', alpha=0.5)
ax[1].bar(x="Test", height=LR.score(x_train,y_train), color='r', alpha=0.5)
ax[1].set_title("Score")

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))
fig.suptitle('Метрики качества', size =16)

ax[0].bar(x="Train dirty",height = 10.775086596878042, color='y', alpha=0.5)
ax[0].bar(x="Train",height = mean_squared_error(y_train, LR.predict(x_train)), color='r', alpha=0.5)
ax[0].bar(x="Test dirty", height= 7.349264129410241, color='y', alpha=0.5)
ax[0].bar(x="Test", height= mean_squared_error(y_test, LR.predict(x_test)), color='r', alpha=0.5)
ax[0].set_title("MSE ")

ax[1].bar(x="Train dirty", height= -0.5296032297616697, color='y', alpha=0.5)
ax[1].bar(x="Train", height= LR.score(x_test,y_test), color='r', alpha=0.5)
ax[1].bar(x="Test dirty", height=0.42878283190980926, color='y', alpha=0.5)
ax[1].bar(x="Test", height=LR.score(x_train,y_train), color='r', alpha=0.5)
ax[1].set_title("Score")

plt.show()

In [ ]:
y_pred = LR.predict(X_test)

In [ ]:
df_pred = pd.DataFrame({'ID': id_col, 'item_cnt_month': y_pred })

df_pred

In [ ]:
df_pred.to_csv('sample_submission.csv', index=False)

# <center>Analysis of data</center>

In [ ]:
df_train = pd.read_csv('sales_train.csv')
df_items = pd.read_csv('items.csv')
df_test = pd.read_csv('test.csv')
df_submission = pd.read_csv('sample_submission.csv')

In [ ]:
y = df_train.groupby(by=["date_block_num"]).sum()['item_cnt_day']
f, ax = plt.subplots(figsize=(8, 6))
sns.distplot(y)

In [ ]:
fig = plt.figure(figsize = (10,6))
res = stats.probplot(y, plot=plt)

In [ ]:
fig = plt.figure(figsize = (10,6))
res = stats.probplot(np.log1p(y), plot=plt)

In [ ]:
f, ax = plt.subplots(figsize=(10, 7))
highcorr_vars = (df_train.corr().item_cnt_day.sort_values(ascending=False)[:10]).index
sns.heatmap(df_train[highcorr_vars].corr(), annot=True)
plt.show()

In [ ]:
from math import ceil

grouped = pd.DataFrame(df_train.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].sum().reset_index())
fig, axes = plt.subplots(nrows=5, ncols=2, sharex=True, sharey=True, figsize=(16,20))
num_graph = 10
id_per_graph = ceil(grouped.shop_id.max() / num_graph)
count = 0
for i in range(5):
    for j in range(2):
        sns.pointplot(x='date_block_num', y='item_cnt_day', hue='shop_id', data=grouped[np.logical_and(count*id_per_graph <= grouped['shop_id'], grouped['shop_id'] < (count+1)*id_per_graph)], ax=axes[i][j])
        count += 1

### <center>Data normalization</center>

In [ ]:
# нормировать данные

scaler = StandardScaler()
scaler.fit(x_train)
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
start_time = time.time()
LR = LinearRegression()
LR.fit(x_test,y_test)
LR_predict_test = LR.predict(x_test)
LR_predict_train = LR.predict(x_train)
LR_test_MAE = mean_absolute_error(y_test, LR_predict_test)
LR_train_MAE = mean_absolute_error(y_train, LR_predict_train)
LR_test_RMSE = mean_squared_error(y_test, LR_predict_test)
LR_train_RMSE = mean_squared_error(y_train, LR_predict_train)
LR_train_score = LR.score(x_train,y_train)
LR_test_score = LR.score(x_test,y_test)


print(f'Train set mse: {LR_train_RMSE:.4f}')
print(f'Test set mse: {LR_test_RMSE:.4f}', )
print()
print(f'Test set score: {LR_test_score:.4f}', )
print(f'Train set score: {LR_train_score:.4f}')
print()
print(f"Test mean absolute error {LR_test_MAE:.4f}")
print(f"Train mean absolute error {LR_train_MAE:.4f}")
print()
LR_time = time.time() - start_time
print(f"Время выполнения расчетов {LR_time:.4f}")

In [ ]:
y_pred = LR.predict(X_test)
df_pred = pd.DataFrame({'ID': id_col, 'item_cnt_month': y_pred })
df_pred

In [ ]:
y_pred = LR.predict(X_test_scaled)
df_pred_norm = pd.DataFrame({'ID': id_col, 'item_cnt_month': y_pred })
df_pred_norm

In [ ]:
# df_pred.to_csv('OUT_LR.csv', index=False)

## <center>Decision tree</center>

In [ ]:
start_time = time.time()
DT = DecisionTreeRegressor()
DT.fit(x_train,y_train)

DT_predict_test = DT.predict(x_test)
DT_predict_train = DT.predict(x_train)
DT_test_MAE = mean_absolute_error(y_test, DT_predict_test)
DT_train_MAE = mean_absolute_error(y_train, DT_predict_train)
DT_test_RMSE = mean_squared_error(y_test, DT_predict_test)
DT_train_RMSE = mean_squared_error(y_train, DT_predict_train)
DT_train_score = DT.score(x_train,y_train)
DT_test_score = DT.score(x_test,y_test)

print(f'Train set mse: {DT_train_RMSE:.4f}')
print(f'Test set mse: {DT_test_RMSE:.4f}', )
print()
print(f'Test set score: {DT_test_score:.4f}', )
print(f'Train set score: {DT_train_score:.4f}')
print()
print(f"Test mean absolute error {DT_test_MAE:.4f}")
print(f"Train mean absolute error {DT_train_MAE:.4f}")
print()
DT_time = time.time() - start_time
print(f"Время выполнения расчетов {DT_time:.4f}")

In [ ]:
y_pred = DT.predict(X_test_scaled)
df_pred = pd.DataFrame({'ID': id_col, 'item_cnt_month': y_pred })

df_pred

## <center>1.1 Number of trees 5</center>
<center>(n_estimators=5)</center>

Without normalization

In [ ]:
start_time = time.time()
RF=RandomForestRegressor(n_estimators=15, random_state=42)
RF.fit(x_train,y_train)

RF_predict_test = RF.predict(x_test)
RF_predict_train = RF.predict(x_train)
RF_test_MAE = mean_absolute_error(y_test, RF_predict_test)
RF_train_MAE = mean_absolute_error(y_train, RF_predict_train)
RF_test_RMSE = mean_squared_error(y_test, RF_predict_test)
RF_train_RMSE = mean_squared_error(y_train, RF_predict_train)
RF_train_score = RF.score(x_train,y_train)
RF_test_score = RF.score(x_test,y_test)

print(f'Train set mse: {RF_train_RMSE:.4f}')
print(f'Test set mse: {RF_test_RMSE:.4f}', )
print()
print(f'Test set score: {RF_test_score:.4f}', )
print(f'Train set score: {RF_train_score:.4f}')
print()
print(f"Test mean absolute error {RF_test_MAE:.4f}")
print(f"Train mean absolute error {RF_train_MAE:.4f}")
print()
RF_time = time.time() - start_time
print(f"Время выполнения расчетов {RF_time:.4f}")


In [ ]:
y_pred = RF.predict(X_test)
df_pred = pd.DataFrame({'ID': id_col, 'item_cnt_month': y_pred })

In [ ]:
# df_pred.to_csv('OUT_RF.csv', index=False)

In [ ]:
df_pred

## <center>Plots</center>

MSE of test and train data according to the number of trees.

In [ ]:
N = [5, 10, 15, 20, 30, 50]
MSE_train = [0.54, 0.43, 0.38, 0.44, 0.39, 0.38]
MSE_test = [1.62, 1.37, 1.20, 1.19, 1.29, 1.35]
MSE_train_scaled = [0.53, 0.43, 0.37, 0.44, 0.40, 0.38]
MSE_test_scaled = [1.63, 1.38, 1.19, 1.47, 1.46, 1.26]

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(N, MSE_train, label="Исходные данные")
plt.plot(N, MSE_train_scaled, label="Нормализованные данные")
plt.title('Среднеквадратическая ошибка \n тренировочных данных')
plt.ylabel("MSE")
plt.xlabel("Количество деревьев")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(N, MSE_test, label="Исходные данные")
plt.plot(N, MSE_test_scaled, label="Нормализованные данные")
plt.title('Среднеквадратическая ошибка \n тестовых данных')
plt.ylabel("MSE")
plt.xlabel("Количество деревьев")
plt.legend()
plt.show()

## <center> Lasso</center>

In [ ]:
start_time = time.time()
lasso=Lasso(alpha=1.0)
lasso.fit(x_train,y_train)

lasso_predict_test = lasso.predict(x_test)
lasso_predict_train = lasso.predict(x_train)
lasso_test_MAE = mean_absolute_error(y_test, lasso_predict_test)
lasso_train_MAE = mean_absolute_error(y_train, lasso_predict_train)
lasso_test_RMSE = mean_squared_error(y_test, lasso_predict_test)
lasso_train_RMSE = mean_squared_error(y_train, lasso_predict_train)
lasso_train_score = lasso.score(x_train,y_train)
lasso_test_score = lasso.score(x_test,y_test)

print(f'Train set mse: {lasso_train_RMSE:.4f}')
print(f'Test set mse: {lasso_test_RMSE:.4f}', )
print()
print(f'Test set score: {lasso_test_score:.4f}', )
print(f'Train set score: {lasso_train_score:.4f}')
print()
print(f"Test mean absolute error {lasso_test_MAE:.4f}")
print(f"Train mean absolute error {lasso_train_MAE:.4f}")
print()
lasso_time = time.time() - start_time
print(f"Время выполнения расчетов {lasso_time:.4f}")

In [ ]:
y_pred = lasso.predict(X_test)
df_pred = pd.DataFrame({'ID': id_col, 'item_cnt_month': y_pred })

df_pred

## <center>Ridge regression</center>

In [ ]:
start_time = time.time()
RD = Ridge(alpha=1.0)
RD.fit(x_train,y_train)

RD_predict_test = RD.predict(x_test)
RD_predict_train = RD.predict(x_train)
RD_test_MAE = mean_absolute_error(y_test, RD_predict_test)
RD_train_MAE = mean_absolute_error(y_train, RD_predict_train)
RD_test_RMSE = mean_squared_error(y_test, RD_predict_test)
RD_train_RMSE = mean_squared_error(y_train, RD_predict_train)
RD_train_score = RD.score(x_train,y_train)
RD_test_score = RD.score(x_test,y_test)

print(f'Train set mse: {RD_train_RMSE:.4f}')
print(f'Test set mse: {RD_test_RMSE:.4f}', )
print()
print(f'Test set score: {RD_test_score:.4f}', )
print(f'Train set score: {RD_train_score:.4f}')
print()
print(f"Test mean absolute error {RD_test_MAE:.4f}")
print(f"Train mean absolute error {RD_train_MAE:.4f}")
print()
RD_time = time.time() - start_time
print(f"Время выполнения расчетов {RD_time:.4f}")

In [ ]:
y_pred = RD.predict(X_test_scaled)
df_pred = pd.DataFrame({'ID': id_col, 'item_cnt_month': y_pred })

df_pred

In [ ]:
# df_pred.to_csv('OUT_Ridge.csv', index=False)

# <center>CatBoost</center>

In [ ]:
start_time = time.time()

cat = cb.CatBoostRegressor(
                                depth = 4,
                                l2_leaf_reg = 1,
                                learning_rate = 0.03)

cat.fit(x_train, y_train)

сat_time = time.time() - start_time

In [ ]:
cat_predict_test = cat.predict(x_test)
cat_predict_train = cat.predict(x_train)
cat_test_MAE = mean_absolute_error(y_test, cat_predict_test)
cat_train_MAE = mean_absolute_error(y_train, cat_predict_train)
cat_test_RMSE = mean_squared_error(y_test, cat_predict_test)
cat_train_RMSE = mean_squared_error(y_train, cat_predict_train)
cat_train_score = cat.score(x_train,y_train)
cat_test_score = cat.score(x_test,y_test)

print(f'Train set mse: {cat_train_RMSE:.4f}')
print(f'Test set mse: {cat_test_RMSE:.4f}', )
print()
print(f'Test set score: {cat_test_score:.4f}', )
print(f'Train set score: {cat_train_score:.4f}')
print()
print(f"Test mean absolute error {cat_test_MAE:.4f}")
print(f"Train mean absolute error {cat_train_MAE:.4f}")
print()

print(f"Время выполнения расчетов {сat_time:.4f}")


In [ ]:
# y_pred = cat.predict(X_test)
# df_pred = pd.DataFrame({'ID': id_col, 'item_cnt_month': y_pred })

# df_pred
# df_pred.to_csv('CatBoost.csv', index=False)

## <center>Summary plots</center>

In [ ]:
index = ["linear\nregression", "Decision\ntree", "Random\nforest", "Lasso", "Ridge-Regression", "CatBoost"]

MSE_train = [LR_train_RMSE, DT_train_RMSE, RF_train_RMSE, lasso_train_RMSE, RD_train_RMSE, cat_train_RMSE]
MSE_test = [LR_test_RMSE, DT_test_RMSE, RF_test_RMSE, lasso_test_RMSE, RD_test_RMSE, cat_test_RMSE]
plt.figure(figsize=(15,10))
plt.title('MSE of different methods')
plt.ylabel("MSE")
plt.xlabel("Methods")

plt.bar(x=index, height=MSE_test, alpha = 0.5, color = "#0000CD", label = "MSE test")
plt.bar(x=index, height=MSE_train, alpha = 0.5, color = "#FFFF00", label = "MSE train")
plt.legend(loc = "best")


plt.show()

In [ ]:
index = ["linear\nregression", "Decision\ntree", "Random\nforest", "Lasso", "Ridge-Regression", "CatBoost"]

MAE_train = [LR_train_MAE, DT_train_MAE, RF_train_MAE, lasso_train_MAE, RD_train_MAE, cat_train_MAE]
MAE_test = [LR_test_MAE, DT_test_MAE, RF_test_MAE, lasso_test_MAE, RD_test_MAE, cat_test_MAE]
plt.figure(figsize=(15,10))
plt.title('MAE of different methods')
plt.ylabel("MАE")
plt.xlabel("Methods")

plt.bar(x=index, height=MAE_test, alpha = 0.5, color = "#DA70D6", label = "MAE test")
plt.bar(x=index, height=MAE_train, alpha = 0.5, color = "#FF4500", label = "MAE train")
plt.legend(loc = "best")


plt.show()

In [ ]:
index = ["linear\nregression", "Decision\ntree", "Random\nforest", "Lasso", "Ridge-Regression", "CatBoost"]

score_train = [LR_train_score, DT_train_score, RF_train_score, lasso_train_score, RD_train_score, cat_train_score]
score_test = [LR_test_score, DT_test_score, RF_test_score, lasso_test_score, RD_test_score, cat_test_score]
plt.figure(figsize=(15,10))
plt.title('Presision score')
plt.ylabel("score")
plt.xlabel("Methods")

plt.bar(x=index, height=score_test, alpha = 0.5, color = "#00FFFF", label = "score test")
plt.bar(x=index, height=score_train, alpha = 0.5, color = "#808080", label = "score train")
plt.legend(loc = "best")


plt.show()

### <center>Time for calculation</center>

In [ ]:
index = ["linear\nregression", "Decision\ntree", "Random\nforest", "Lasso", "Ridge-Regression", "CatBoost"]

work_time = [LR_time, DT_time, RF_time, lasso_time, RD_time, сat_time]
# MSE_test = [2.02, 1.37, 2.32, 2.02]
plt.figure(figsize=(15,10))
plt.title('Time for calculation')
plt.ylabel("Time, sec")
plt.xlabel("Methods")

plt.bar(x=index, height=work_time, alpha = 0.5, color = "#0000CD")
# plt.bar(x=index, height=MSE_train, alpha = 0.5, color = "#FFFF00", label = "MSE тренировочных данных")
plt.legend(loc = "best")

plt.show()